<a href="https://colab.research.google.com/github/toskn/thesis/blob/main/conll2spanbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install jsonlines

In [2]:
# маунтим гугл диск со всеми данными
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import jsonlines

In [4]:
# залезаем в нашу папку
%cd /content/drive/MyDrive/RuCor2SpanBert

/content/drive/MyDrive/RuCor2SpanBert


In [ ]:
# поправленные реквайрментсы подключаем
!pip install -r requirements.txt

In [ ]:
!mkdir data

In [ ]:
!mkdir Data

Для запуска необходимо задать **data_dir**

In [6]:
import os
os.environ['data_dir'] = "./data"
os.environ['ontonotes_path'] = "-"
#os.environ['CHOSEN_MODEL'] = 'spanbert_large'

Some `g++` versions may not play nicely with this repo. If you get this:
`tensorflow.python.framework.errors_impl.NotFoundError: ./coref_kernels.so: undefined symbol: _ZN10tensorflow12OpDefBuilder4AttrESs`, try removing the flag `-D_GLIBCXX_USE_CXX11_ABI=0` from `setup_all.sh`. Thanks to Naman Jain for the [solution](https://github.com/mandarjoshi90/coref/issues/29).


In [7]:
# дадим права файлу ниже, чтобы он запускался 
!chmod 755 -R /content/drive/MyDrive/RuCor2SpanBert/setup_all.sh

Thanks to stackoverflow for the [solution](https://stackoverflow.com/questions/64173936/why-am-i-getting-permission-denied-error-in-google-colab).


In [ ]:
! ./setup_all.sh

# Подготовка CoNLL файлов в jsonlines 

In [9]:
!pwd

/content/drive/My Drive/RuCor2SpanBert


In [10]:
# положите <train, test, dev>.english.v4_gold_conll в Data. пока такое название хоть там и русский язык внутри.

In [11]:
import re
import os
import sys
import json
import tempfile
import subprocess
import collections

import util as util
import conll as conll
from bert import tokenization

from minimize import minimize_language


vocab_file = './cased_config_vocab/vocab.txt'
input_dir = './Data'
output_dir = './Output'
do_lower_case = False # True if 'chinese' in vocab_file else False

print('Lower case = ' + str(do_lower_case)) 
      
labels = collections.defaultdict(set)
stats = collections.defaultdict(int)

if not os.path.isdir(output_dir):
  os.mkdir(output_dir)

for seg_len in [256, 384]: # 128, 256, 384 …
  minimize_language("english", labels, stats, vocab_file, seg_len, input_dir, output_dir, do_lower_case)
  # minimize_language("chinese", labels, stats, vocab_file, seg_len)
  # minimize_language("es", labels, stats, vocab_file, seg_len)
  # minimize_language("arabic", labels, stats, vocab_file, seg_len)

for k, v in labels.items():
  print("{} = [{}]".format(k, ", ".join("\"{}\"".format(label) for label in v)))
for k, v in stats.items():
  print("{} = {}".format(k, v))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Lower case = False
Minimizing ./Data/dev.english.v4_gold_conll
Wrote 1843 documents to ./Output/dev.english.256.jsonlines
Minimizing ./Data/train.english.v4_gold_conll
Wrote 67769 documents to ./Output/train.english.256.jsonlines
Minimizing ./Data/test.english.v4_gold_conll
Wrote 1749 documents to ./Output/test.english.256.jsonlines
Minimizing ./Data/dev.english.v4_gold_conll
Wrote 1843 documents to ./Output/dev.english.384.jsonlines
Minimizing ./Data/train.english.v4_gold_conll
Wrote 67769 documents to ./Output/train.english.384.jsonlines
Minimizing ./Data/test.english.v4_gold_conll
Wrote 1749 documents to ./Output/test.english.384.jsonlines
max_sent_len_english = 384


In [9]:
# создаем файл для трейна

i = 0

with jsonlines.open('./Output/train.english.256.jsonlines') as reader:

  with jsonlines.open('./data/train.english.256.jsonlines', mode='w') as writer:

    for o in reader:

      # выберем первые 20к из 68к для тренировки
      if i >= 0:
        o['doc_key'] = 'wb'
        writer.write(o)
      if i == 20000:
        break

      i += 1

# Запуск тренировки модели

In [12]:
! chmod 755 -R /content/drive/MyDrive/RuCor2SpanBert/setup_training.sh

In [ ]:
! ./setup_training.sh - $data_dir

In [9]:
! chmod 755 -R /content/drive/MyDrive/RuCor2SpanBert/conll-2012/scorer/v8.01/scorer.pl

In [ ]:
! GPU=0 python train.py train_spanbert_base

# Запуск предсказаний


In [4]:
! chmod 755 -R /content/drive/MyDrive/RuCor2SpanBert/download_pretrained.sh

In [ ]:
! ./download_pretrained.sh spanbert_base

In [ ]:
! GPU=0 python predict.py spanbert_base data/dev.english.256.jsonlines dev_evaluation_12m.jsonlines

In [ ]:
!pwd

In [ ]:
# поменял эксперимент спанберт база количество сегментов на 512 и имя файла на 512

In [11]:
# подготовка файла обработанного токенайзером 
# убираем кластеры и собираем только нужные 
# для предсказания данные в jsonlines
with jsonlines.open('./Output/test.english.256.jsonlines') as reader:

  with jsonlines.open('./data/test.english.256.jsonlines', mode='w') as writer:

    eval_line = {}

    for o in reader:

      o['doc_key'] = 'wb'
      o['clusters'] = []

      eval_line.update({'clusters': o['clusters']})
      eval_line.update({'doc_key': o['doc_key']})
      eval_line.update({'sentences': o['sentences']})
      eval_line.update({'speakers': o['speakers']})
      eval_line.update({'sentence_map': o['sentence_map']})
      eval_line.update({'subtoken_map': o['subtoken_map']})

      writer.write(eval_line)

# Результаты предсказаний

In [12]:
with jsonlines.open('dev_evaluation_12m.jsonlines') as reader:
  x = [o for o in reader]

In [13]:
len(x)

1843

In [14]:
def convert_mention(output, mention, comb_text):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

def get_clusters(output):
    comb_text = [word for sentence in output['sentences'] for word in sentence]
    seen = set()
    #print('Clusters:')
    clusters = []
    for cluster in output['predicted_clusters']:
        mapped = []
        for mention in cluster:
            seen.add(tuple(mention))
            mapped.append(convert_mention(output, mention, comb_text))
        clusters.append(mapped)

    return clusters

In [15]:
predictions = [get_clusters(output) for output in x]

In [16]:
clusters = [[[m[1] for m in cluster] for cluster in clusters] for clusters in predictions]

In [17]:
for element in clusters:
  if element != []:
    print(element)

[['сыну', 'нам']]
[['сына', 'нам']]
[['муж', 'мужа']]
[['сыну', 'мы']]
[['мы', 'мы', 'Мы'], ['дочь', 'дочь']]
[['меня', 'меня', 'меня', 'меня']]
[['мне', 'Меня', 'мне', 'мной'], ['мама', 'мама', 'отец', 'неё', 'мамы']]
[['нас', 'нам', 'нам', 'мы'], ['меня', 'меня']]
[['Мы', 'нас', 'нас', 'нас', 'Нас', 'нас', 'нас', 'нас', 'мы', 'нас']]
[['мне', 'меня', 'мне', 'мне', 'меня', 'мне']]
[['Сын', 'сыну']]
[['Мне', 'меня']]
[['мужа', 'нам']]
[['Мама', 'мама']]
[['Мама', 'Мама', 'мамы', 'маме', 'Мама']]
[['меня', 'меня']]
[['сына', 'сына']]
[['меня', 'меня'], ['мы', 'мы']]
[['нее', 'нее']]
[['Меня', 'меня'], ['сына', 'сына', 'сын']]
[['меня', 'Мне', 'меня', 'мне', 'мне', 'мне']]
[['нам', 'нам', 'Мы']]
[['меня', 'меня', 'меня', 'меня', 'меня', 'мне']]
[['Меня', 'мне', 'мне', 'мне', 'мне', 'Мне', 'мне', 'Меня', 'мне']]
[['маме', 'мама', 'она']]
[['меня', 'мне', 'меня']]
[['меня', 'мне']]
[['мне', 'меня', 'мне']]
[['мне', 'мне', 'мне']]
[['сына', 'нам', 'мы']]
[['нам', 'мы', 'нас', 'мы', 'мы', 'н